In [4]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
import tensorflow as tf

In [9]:
X = np.array([8.])
B = np.array([2.])
model = Sequential()
model.add(Dense(input_shape=(1,), bias_initializer="ones", units=1))

model.compile(loss='mae', optimizer='adam')
model.fit(X, B, nb_epoch=100,batch_size=1)

Epoch 1/100
1/1 [==============================] - 0s 55ms/step - loss: 1.8726
Epoch 2/100
1/1 [==============================] - 0s 1ms/step - loss: 1.8659
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 1.8569
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 1.8479
Epoch 5/100
1/1 [==============================] - 0s 1ms/step - loss: 1.8389
Epoch 6/100
1/1 [==============================] - 0s 1ms/step - loss: 1.8299
Epoch 7/100
1/1 [==============================] - 0s 839us/step - loss: 1.8209
Epoch 8/100
1/1 [==============================] - 0s 990us/step - loss: 1.8119
Epoch 9/100
1/1 [==============================] - 0s 1ms/step - loss: 1.8029
Epoch 10/100
1/1 [==============================] - 0s 1ms/step - loss: 1.7939
Epoch 11/100
1/1 [==============================] - 0s 1ms/step - loss: 1.7849
Epoch 12/100
1/1 [==============================] - 0s 865us/step - loss: 1.7759
Epoch 13/100
1/1 [==============================] - 0s

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  import sys


Epoch 21/100
1/1 [==============================] - 0s 2ms/step - loss: 1.6949
Epoch 22/100
1/1 [==============================] - 0s 4ms/step - loss: 1.6859
Epoch 23/100
1/1 [==============================] - 0s 4ms/step - loss: 1.6769
Epoch 24/100
1/1 [==============================] - 0s 5ms/step - loss: 1.6679
Epoch 25/100
1/1 [==============================] - 0s 2ms/step - loss: 1.6589
Epoch 26/100
1/1 [==============================] - 0s 2ms/step - loss: 1.6499
Epoch 27/100
1/1 [==============================] - 0s 5ms/step - loss: 1.6409
Epoch 28/100
1/1 [==============================] - 0s 1ms/step - loss: 1.6319
Epoch 29/100
1/1 [==============================] - 0s 2ms/step - loss: 1.6229
Epoch 30/100
1/1 [==============================] - 0s 8ms/step - loss: 1.6139
Epoch 31/100
1/1 [==============================] - 0s 2ms/step - loss: 1.6049
Epoch 32/100
1/1 [==============================] - 0s 3ms/step - loss: 1.5959
Epoch 33/100
1/1 [==============================] - 

/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.input

<tf.Tensor 'embedding_1_input:0' shape=(?, ?) dtype=float32>

In [22]:
model.output

<tf.Tensor 'dense_4/Sigmoid:0' shape=(?, 1) dtype=float32>

In [23]:
model.layers

In [24]:
outputTensor = model.output
listOfVariableTensors = model.inputs[0]
gradients = tf.gradients(outputTensor, listOfVariableTensors)
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
evaluated_gradients = sess.run(gradients,feed_dict={model.input:X[0].reshape(-1,1)})

/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


TypeError: Fetch argument None has invalid type <class 'NoneType'>

In [11]:
model.layers[0].get_weights()

[array([[-0.10907829]], dtype=float32), array([1.], dtype=float32)]

In [12]:
evaluated_gradients

[array([[-0.10907829]], dtype=float32)]

# Gradient Test

In [18]:
import tensorflow as tf
from random import randint

dims = 8
pos  = randint(0, dims - 1)

logits = tf.random_uniform([dims], maxval=3, dtype=tf.float32)
labels = tf.one_hot(pos, dims)

res1 = tf.nn.softmax_cross_entropy_with_logits(       logits=logits, labels=labels)
res2 = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=tf.constant(pos))

with tf.Session() as sess:
    a, b = sess.run([res1, res2])
    print (a, b)
    print (a == b)

W0729 12:09:24.049191 4500092352 deprecation.py:323] From <ipython-input-18-5d5219640aa9>:10: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



2.4087439 2.4087439
True


# IMDB Test

In [19]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Build model...


W0805 17:10:54.816534 4500092352 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0805 17:10:55.084730 4500092352 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/15
25000/25000 [==============================] - 95s 4ms/step - loss: 0.4554 - acc: 0.7858 - val_loss: 0.3998 - val_acc: 0.8345
Epoch 2/15
25000/25000 [==============================] - 94s 4ms/step - loss: 0.3030 - acc: 0.8770 - val_loss: 0.4129 - val_acc: 0.8139
Epoch 3/15
25000/25000 [==============================] - 94s 4ms/step - loss: 0.2233 - acc: 0.9137 - val_loss: 0.4248 - val_acc: 0.8323
Epoch 4/15
25000/25000 [==============================] - 91s 4ms/step - loss: 0.1550 - acc: 0.9430 - val_loss: 0.4551 - val_acc: 0.8270
Epoch 5/15
25000/25000 [==============================] - 90s 4ms/step - loss: 0.1160 - acc: 0.9580 - val_loss: 0.5870 - val_acc: 0.8264
Epoch 6/15
25000/25000 [==============================] - 89s 4ms/step - loss: 0.0879 - acc: 0.9686 - val_loss: 0.5921 - val_acc: 0.8249
Epoch 7/15
25000/25000 [==============================] - 91s 4ms/step - loss: 0.0565 - acc: 0.9810 - val_loss: 0.7263 

In [42]:
model.trainable_weights

[<tf.Variable 'embedding_1/embeddings:0' shape=(20000, 128) dtype=float32_ref>,
 <tf.Variable 'lstm_1/kernel:0' shape=(128, 512) dtype=float32_ref>,
 <tf.Variable 'lstm_1/recurrent_kernel:0' shape=(128, 512) dtype=float32_ref>,
 <tf.Variable 'lstm_1/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'dense_4/kernel:0' shape=(128, 1) dtype=float32_ref>,
 <tf.Variable 'dense_4/bias:0' shape=(1,) dtype=float32_ref>]

In [44]:
outputTensor = model.output
listOfVariableTensors = model.trainable_weights
gradients = tf.gradients(outputTensor, listOfVariableTensors)

(80, 1)

In [53]:
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
evaluated_gradients = sess.run(gradients,feed_dict={model.input:x_train[0].reshape(-1,1)})

/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [56]:
x_train[0]

array([   15,   256,     4,     2,     7,  3766,     5,   723,    36,
          71,    43,   530,   476,    26,   400,   317,    46,     7,
           4, 12118,  1029,    13,   104,    88,     4,   381,    15,
         297,    98,    32,  2071,    56,    26,   141,     6,   194,
        7486,    18,     4,   226,    22,    21,   134,   476,    26,
         480,     5,   144,    30,  5535,    18,    51,    36,    28,
         224,    92,    25,   104,     4,   226,    65,    16,    38,
        1334,    88,    12,    16,   283,     5,    16,  4472,   113,
         103,    32,    15,    16,  5345,    19,   178,    32],
      dtype=int32)

In [59]:
evaluated_gradients[0]

IndexedSlicesValue(values=array([[ 2.7242852e-03,  4.0505431e-03,  1.1173676e-04, ...,
        -8.9879363e-04,  1.4633462e-03, -7.0275259e-03],
       [ 2.9114098e-03,  4.2174039e-03,  2.3876190e-04, ...,
        -9.3307858e-04,  1.3719096e-03, -7.0317956e-03],
       [ 2.8904041e-03,  4.0378273e-03,  4.7502435e-05, ...,
        -9.1432547e-04,  1.5129835e-03, -7.0051588e-03],
       ...,
       [ 2.7991657e-03,  3.8761070e-03,  9.7190714e-05, ...,
        -8.6819183e-04,  1.3717442e-03, -7.0829610e-03],
       [ 2.8634320e-03,  4.0161214e-03,  5.4811720e-05, ...,
        -9.4301958e-04,  1.3748364e-03, -7.1229725e-03],
       [ 2.7674010e-03,  3.9366805e-03, -1.2644870e-05, ...,
        -9.5367950e-04,  1.3547722e-03, -6.9717499e-03]], dtype=float32), indices=array([   15,   256,     4,     2,     7,  3766,     5,   723,    36,
          71,    43,   530,   476,    26,   400,   317,    46,     7,
           4, 12118,  1029,    13,   104,    88,     4,   381,    15,
         297,    98